In [29]:
!pip install openpyxl pandas transformers accelerate

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import pandas as pd

df = pd.read_excel('Traits List.xlsx')
essential_traits = [
    'Active', 'Adventurous', 'Affectionate', 'Ambitious', 'Angry', 'Anxious', 'Arrogant',
    'Attentive', 'Bossy', 'Brave', 'Calm', 'Capable', 'Careful', 'Cautious', 'Charismatic',
    'Cheerful', 'Clever', 'Clumsy', 'Cold-Hearted', 'Compassionate', 'Confident', 'Considerate',
    'Cooperative', 'Courageous', 'Cowardly', 'Critical', 'Cruel', 'Curious', 'Daring',
    'Decisive', 'Dependable', 'Determined', 'Diligent', 'Dishonest', 'Disrespectful',
    'Eager', 'Easygoing', 'Efficient', 'Eloquent', 'Embarrassed', 'Energetic', 'Enthusiastic',
    'Fair', 'Faithful', 'Fearless', 'Friendly', 'Funny', 'Generous', 'Gentle', 'Grateful',
    'Greedy', 'Grouchy', 'Gullible', 'Happy', 'Helpful', 'Honest', 'Hopeful', 'Humble',
    'Impulsive', 'Independent', 'Innocent', 'Intelligent', 'Jealous', 'Kind', 'Lazy',
    'Logical', 'Lonely', 'Loving', 'Loyal', 'Mean', 'Mature', 'Moody', 'Naïve', 'Nervous',
    'Obedient', 'Optimistic', 'Organized', 'Pessimistic', 'Polite', 'Popular', 'Positive',
    'Proud', 'Reliable', 'Respectful', 'Responsible', 'Rude', 'Sarcastic', 'Selfish',
    'Sensitive', 'Shy', 'Smart', 'Sociable', 'Stubborn', 'Sweet', 'Talkative', 'Thoughtful',
    'Timid', 'Trustworthy', 'Wise', 'Witty'
]

df = df[df['Trait'].isin(essential_traits)].reset_index(drop=True)
df.head()

,Trait,Description
0,Active,Tending to move around often; full of energy
1,Adventurous,Willing to take risks; likes new adventures & ...
2,Affectionate,Showing affection; tender; loving
3,Ambitious,Eager to succeed; full of desire
4,Angry,Feeling or showing anger


In [28]:
df.shape

(91, 2)

In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

system_prompt_template = """
You are given a personality trait and its description. Perform the following tasks:

1. Generate a system prompt that would cause an LLM to simulate this trait in its responses. It should follow this format:
System Prompt: [text that guides LLM behavior in line with the trait]

2. Translate the trait name into:
• Russian: [Trait in Russian]
• Ukrainian: [Trait in Ukrainian]

3. Classify the sentiment of the trait as one of:
• Positive
• Negative
• Neutral

Respond in this format exactly:

Trait: [original trait name]
System Prompt: [system prompt text]
Russian: [translated trait]
Ukrainian: [translated trait]
Sentiment: [Positive/Negative/Neutral]

Trait: {trait}
Description: {description}
"""

def generate_answer(trait, description):
    prompt = system_prompt_template.format(trait=trait, description=description)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")
    output = model.generate(**inputs, max_new_tokens=250, do_sample=True, temperature=0.7)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

results = []

for _, row in df.iterrows():
    trait = row['trait']
    description = row['description']
    response = generate_answer(trait, description)
    try:
        trait_name = response.split("Trait:")[1].split("\n")[0].strip()
        system_prompt = response.split("System Prompt:")[1].split("Russian:")[0].strip()
        russian = response.split("Russian:")[1].split("Ukrainian:")[0].strip()
        ukrainian = response.split("Ukrainian:")[1].split("Sentiment:")[0].strip()
        sentiment = response.split("Sentiment:")[1].strip()
    except IndexError as e:
        print(e)
        continue

    # Добавление в список
    results.append({
        "Trait": trait_name,
        "System Prompt": system_prompt,
        "Russian": russian,
        "Ukrainian": ukrainian,
        "Sentiment": sentiment
    })

# Создание DataFrame
df_output = pd.DataFrame(results)
df_output.head()

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import json

alpaca_df = pd.read_json("alpaca_data.json")
df_output = pd.read_csv("traits_with_prompts.csv")

N = 100

final_dataset = []
for _, row in df_output.iterrows():
    trait = row["Trait"]
    system_prompt = row["System Prompt"]
    instructions = alpaca_df.sample(N, random_state=42)["instruction"].tolist()
    
    for instr in instructions:
        full_prompt = f"<|system|>\n{system_prompt}\n<|user|>\n{instr}\n<|assistant|>\n"
        entry = {
            "trait": trait,
            "system_prompt": system_prompt,
            "alpaca_prompt": instr,
            "full_prompt": full_prompt
        }
        final_dataset.append(entry)

with open("trait_combined_dataset.json", "w", encoding="utf-8") as f:
    json.dump(final_dataset, f, ensure_ascii=False, indent=2)